In [6]:
import pandas as pd
import seaborn as sns

In [2]:
NUTRITION_COLS = ['area_id', 'carb', 'fibre', 'protein', 'sugar']

year_grocery = pd.read_csv("data/year_lsoa_grocery.csv")[NUTRITION_COLS]
print(year_grocery.shape)
display(year_grocery.head())

(4833, 5)


,area_id,carb,fibre,protein,sugar
0,E01000001,15.158014,1.622653,5.262429,9.213734
1,E01000002,14.358466,1.692822,5.351774,8.337412
2,E01000003,15.820254,1.522523,5.029519,9.414937
3,E01000005,17.126487,1.612862,5.230254,9.603258
4,E01000006,19.903063,1.640227,5.026295,11.355115


In [3]:
COLS = ["Code", "Year ending Dec 2014"]

housing_prices = pd.read_excel(
    "data/land-registry-house-prices-LSOA.xls", sheet_name="Mean")[COLS]
housing_prices.rename(columns={"Year ending Dec 2014": "mean house prices"}, inplace=True)
housing_prices.dropna(inplace=True)
print(housing_prices.shape)
display(housing_prices.head())

(4835, 2)


,Code,mean house prices
1,E01000001,853613
2,E01000002,1.02526e+06
3,E01000003,598329
4,E01000005,417325
5,E01032739,906953


In [9]:
grocery_housing = pd.merge(year_grocery, housing_prices, left_on="area_id", right_on="Code")
grocery_housing.drop("Code", axis=1, inplace=True)
display(grocery_housing)

,area_id,carb,fibre,protein,sugar,mean house prices
0,E01000001,15.158014,1.622653,5.262429,9.213734,853613
1,E01000002,14.358466,1.692822,5.351774,8.337412,1.02526e+06
2,E01000003,15.820254,1.522523,5.029519,9.414937,598329
3,E01000005,17.126487,1.612862,5.230254,9.603258,417325
4,E01000006,19.903063,1.640227,5.026295,11.355115,249441
...,...,...,...,...,...,...
4828,E01033742,17.065225,1.551666,5.384490,9.149620,226944
4829,E01033743,19.474750,1.504327,4.937452,10.940879,202000
4830,E01033744,19.164089,1.524306,5.309611,10.220282,228045
4831,E01033745,19.682040,1.598945,5.160060,10.230521,208148


In [13]:
display(grocery_housing.columns)
display(grocery_housing.corr())

sns.heatmap(grocery_housing.corr()[['mean house prices']])

Index(['area_id', 'carb', 'fibre', 'protein', 'sugar', 'mean house prices'], dtype='object')

,carb,fibre,protein,sugar
carb,1.000000,0.173675,-0.081434,0.706033
fibre,0.173675,1.000000,0.258154,0.032011
protein,-0.081434,0.258154,1.000000,-0.360883
sugar,0.706033,0.032011,-0.360883,1.000000


KeyError: "None of [Index(['mean house prices'], dtype='object')] are in the [columns]"